# AI Tool for Code Optimization



In [13]:
!pip install transformers tokenizers line-profiler memory-profiler gradio flake8

Snippet that can be used to check

```
def filter_even_numbers():
    numbers = [i for i in range(1, 1001)]
    even_numbers = [num for num in numbers if num % 2 == 0]
    return even_numbers

filter_even_numbers()
```





```
def string_concatenation_example():
    result = ""
    for i in range(10000):
        result += str(i)
    return result

print(string_concatenation_example())
```



In [14]:
from typing import Tuple
import os
import tempfile
from io import StringIO
import cProfile
import pstats
from line_profiler import LineProfiler
from memory_profiler import memory_usage
import tracemalloc
from functools import lru_cache
from transformers import AutoModelForCausalLM, AutoTokenizer
import gradio as gr

# Load optimized CodeGen model and tokenizer
model_name = "codeparrot/codeparrot-small"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to detect errors and suggest fixes using CodeGen
def detect_and_correct_code(code: str) -> str:
    inputs = tokenizer(code, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=150)
    corrected_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_code

# Profiling functions

def profile_code_with_line_profiler(code: str) -> str:
    profiler = LineProfiler()
    try:
        exec(code)
    except Exception as e:
        return f"Error executing code: {str(e)}"
    profiler.enable_by_count()
    exec(code)  # You can optimize by reducing repetitive execution here
    profiler.disable_by_count()
    s = StringIO()
    profiler.print_stats(stream=s)
    return s.getvalue()

def profile_code_with_cprofile(code: str) -> str:
    profiler = cProfile.Profile()
    profiler.enable()
    try:
        exec(code)
    except Exception as e:
        return f"Error executing code: {str(e)}"
    profiler.disable()
    s = StringIO()
    ps = pstats.Stats(profiler, stream=s)
    ps.sort_stats('cumulative').print_stats()
    return s.getvalue()

# Static code analysis function using flake8
def static_code_analysis_with_flake8(code: str) -> str:
    # Add your Flake8 logic here
    pass

# Analyze performance and suggest optimizations
@lru_cache(maxsize=32)
def analyze_performance(code: str) -> Tuple[str, str, str, str, str, str]:
    if not code.strip():
        return "Please provide some code to analyze.", "", "", "", "", ""

    # Start profiling
    profile_output = profile_code_with_line_profiler(code)
    cprofile_output = profile_code_with_cprofile(code)
    memory_output = profile_memory_with_tracemalloc(code)
    flake8_output = static_code_analysis_with_flake8(code)
    library_suggestions = suggest_libraries(code)
    corrected_code = detect_and_correct_code(code)

    # Real-time feedback for user
    real_time_feedback = f"### Real-Time Feedback:\n"
    real_time_feedback += "**Code Profiling Insights:**\n"
    real_time_feedback += "1. Line Profiler Report: Identifies which lines in your code take the longest to execute.\n"
    real_time_feedback += "2. CProfile Report: Shows function-level performance data.\n"
    real_time_feedback += "3. Memory Usage: Analyzes memory consumption during execution.\n"
    real_time_feedback += "4. Static Code Analysis: Check for errors, warnings, or inefficiencies.\n"
    real_time_feedback += f"**Issues Detected in Your Code:**\n{flake8_output}\n"

    # Description of code optimizations
    optimization_explanation = """
    ### How the Corrected Code is Better:

    1. **Performance Improvements:**
    The corrected code has been optimized for performance by removing unnecessary operations, reducing time complexity, and simplifying logic.

    2. **Memory Efficiency:**
    Memory usage has been reduced by optimizing data structures, avoiding unnecessary allocations, and streamlining variable handling.

    3. **Readability and Maintainability:**
    The AI-corrected code uses clearer variable names, better structure, and follows best coding practices for readability. This makes the code easier to maintain and extend.
    """

    return profile_output, cprofile_output, memory_output, flake8_output, library_suggestions, corrected_code, real_time_feedback, optimization_explanation

# Gradio interface function
def gradio_interface(code_snippet: str) -> Tuple[str, str, str, str, str, str, str, str]:
    profile_output, cprofile_output, memory_output, flake8_output, library_suggestions, corrected_code, real_time_feedback, optimization_explanation = analyze_performance(code_snippet)

    return (
        f"### **Line Profiler Report**\n{profile_output}",
        f"### **CProfile Report**\n{cprofile_output}",
        f"### **Memory Usage**\n{memory_output}",
        f"### **Static Code Analysis (Flake8)**\n{flake8_output}",
        f"### **Optimization Suggestions**\n{library_suggestions}",
        f"### **Corrected Code**\n```python\n{corrected_code}\n```",
        real_time_feedback,
        optimization_explanation
    )

# Gradio Interface
iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(
        lines=10,
        placeholder="Enter Python code here...",
        label="Input Code"
    ),
    outputs=[
        gr.Markdown(label="Line Profiler Report"),
        gr.Markdown(label="CProfile Report"),
        gr.Markdown(label="Memory Usage"),
        gr.Markdown(label="Static Analysis (Flake8)"),
        gr.Markdown(label="Optimization Suggestions"),
        gr.Markdown(label="Corrected Code"),
        gr.Markdown(label="Real-Time Feedback"),
        gr.Markdown(label="Optimization Explanation"),
    ],
    title="Enhanced AI Code Profiler and Optimizer",
    description="""Analyze and optimize your Python code:

    - **Profiling Reports**: Line-by-line and overall performance analysis.
    - **Memory Usage**: Peak memory usage details.
    - **Static Analysis**: Flake8-based error and warning checks.
    - **Optimization Suggestions**: Recommendations for improving code efficiency.
    - **Corrected Code**: AI-generated fixes or optimizations."""
)

# Launch the Gradio interface
iface.launch()



Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://73f9a75b57027b8845.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


---

### **Code Explanation**

- **Model Initialization**:
  - Loads a pre-trained **CodeGen** model from Hugging Face to correct and optimize code snippets.

- **`detect_and_correct_code` Function**:
  - Uses the model to generate corrected versions of the provided code by identifying errors and offering suggestions.

- **`profile_code_with_line_profiler` Function**:
  - Profiles the code line-by-line to identify which lines take the most time to execute, helping to spot performance bottlenecks.

- **`profile_code_with_cprofile` Function**:
  - Uses **CProfile** to profile the overall function-level performance of the code, showing which functions consume the most resources.

- **`static_code_analysis_with_flake8` Function**:
  - Intended to run **flake8**, a static code analysis tool, to check for syntax errors, warnings, or inefficiencies (currently a placeholder).

- **`analyze_performance` Function**:
  - Combines the profiling and analysis functions to provide detailed insights into code performance, memory usage, and error detection.

- **`gradio_interface` Function**:
  - Defines the user interface with **Gradio**, allowing users to input code and receive feedback on performance, suggestions, and AI-generated corrections.

- **Gradio Interface**:
  - Sets up and launches the interactive Gradio interface to display profiling results, optimization suggestions, and corrected code.

---

### **Personal Experience with Gradio & Hugging Face**

I explored **Gradio** and **Hugging Face** during my participation in the **Microsoft AI Learning Hackathon**. Gradio helped me create an intuitive user interface, while Hugging Face’s **CodeGen** model enabled AI-driven code analysis and optimization.

---

